# EURO Simulation

**Summary**: This past month we were withness to the sporting spectecal that is the EUROs. It is an event that takes place every 4 years (5 years this time cause COVID) where the best Countries qualify to compete. In this notebook we will be looking at the round of 16 onwards. Using the results in the group stage to predict the winner of the tournament. 

The first step is to import the data and we will use that to calculate the weights that will be used to calculate the rest of the results. 

For this we will look at Wins, Loses, Draws, Goals For and Goals Against and the formula for calculating weights will be
sum(W_i*x_i)/sum(W) where W are the weights and x is the value of said weight. Where the weights are valued at 
Win = 1 Lose = -1 Draw = 0.5 Goals For = 0.25 Goals Against = -0.25. Wins are more important than loses and both are more important than draws. And the result of the match is more important than the actual goals but teams are still punished for heavy concedings. 

In [7]:
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pennylane as qml
import sklearn.datasets
import sklearn.decomposition
import torch
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import requests

In [54]:
def weightFunc(row):
    return row['Won'] - row['Lost'] + row['Draw']*0.5 +row['GF']*.25 - row['A']*.25

def whoWon(value):
    v_list = value.split(' - ')
    if v_list[0] == v_list[1]:
        return 0.5
    return v_list.index(max(v_list))

In [36]:
df = pd.read_csv('prem_table_2020.csv')
df.keys()
df['Weight'] =df.apply (lambda row: weightFunc(row),axis=1)
df = df[['Team','Weight']]
df = df.replace(to_replace ='Manchester City', value ='Man City')
df = df.replace(to_replace ='Leicester City',value ='Leicester')
df = df.replace('Sheffield United',value ='Sheffield Utd')
df = df.replace('Norwich City',value ='Norwich')
df = df.replace('Wolverhampton Wanderers',value ='Wolves')
df = df.replace('West Ham United',value ='West Ham')
df = df.replace('Tottenham Hotspur',value ='Spurs')
df = df.replace('Newcastle United',value ='Newcastle')
df = df.replace('Manchester United',value ='Man Utd')
df = df.replace('Brighton and Hove Albion',value ='Brighton')
df

,Team,Weight
0,Liverpool,37.00
1,Man City,20.50
2,Leicester,18.50
3,Chelsea,8.75
4,Spurs,8.75
5,Sheffield Utd,8.50
6,Wolves,8.75
7,Everton,2.00
8,Man Utd,6.75
9,Arsenal,8.00


In [61]:
df_matches = pd.read_csv('epl-2019-GMTStandardTime.csv')
df_matches = df_matches[['Home Team','Away Team','Result']]
teams = df_matches['Home Team'].unique()

In [62]:
df_matches['y'] = df_matches['Result'].apply (lambda value: whoWon(value))
for team in teams:
    index = df.index
    cond = df['Team'] == team
    i = index[cond].tolist()[0]
    df_matches = df_matches.replace(team,df['Weight'][i])
df_matches['x'] = df_matches[['Home Team', 'Away Team']].values.tolist()
df_matches = df_matches[['x','y']]
df_matches

,x,y
0,"[37.0, -15.0]",0.0
1,"[-7.25, 20.5]",1.0
2,"[-8.0, 8.5]",0.5
3,"[-2.25, -6.0]",0.0
4,"[-0.75, 2.0]",0.5
...,...,...
375,"[18.5, 6.75]",1.0
376,"[20.5, -15.0]",0.0
377,"[-3.5, 37.0]",1.0
378,"[-6.0, 8.5]",0.0


In [64]:
n_features = 2
n_classes = 2
n_samples = 380
x = df_matches['x'].values.tolist()
y = df_matches['y'].values.tolist()

In [65]:
np.random.seed(1967)
x, y = zip(*np.random.permutation(list(zip(x, y))))

pca = sklearn.decomposition.PCA(n_components=n_features)
pca.fit(x)
x = pca.transform(x)

/Users/williamngana/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


In [66]:
x_min = np.min(x, axis=0)
x_max = np.max(x, axis=0)

x = 2 * np.pi * (x - x_min) / (x_max - x_min) - np.pi

In [68]:
split = int(n_samples*90)

x_train = x[:split]
x_test = x[split:]
y_train = y[:split]
y_test = y[split:]

In [70]:
colours = ["#ec6f86", "#4573e7"]
def plot_points(x_train, y_train, x_test, y_test):
    c_train = []
    c_test = []

    for y in y_train:
        c_train.append(colours[y])

    for y in y_test:
        c_test.append(colours[y])

    plt.scatter(x_train[:, 0], x_train[:, 1], c=c_train)
    plt.scatter(x_test[:, 0], x_test[:, 1], c=c_test, marker="x")

    plt.xlabel("Feature 1", fontsize=16)
    plt.ylabel("Feature 2", fontsize=16)

    ax = plt.gca()
    ax.set_aspect(1)

    c_transparent = "#00000000"

    custom_lines = [
        Patch(facecolor=colours[0], edgecolor=c_transparent, label="Class 0"),
        Patch(facecolor=colours[1], edgecolor=c_transparent, label="Class 1"),
        Patch(facecolor=colours[2], edgecolor=c_transparent, label="Class 2"),
        Line2D([0], [0], marker="o", color=c_transparent, label="Train",
               markerfacecolor="black", markersize=10),
        Line2D([0], [0], marker="x", color=c_transparent, label="Test",
               markerfacecolor="black", markersize=10),
    ]

    ax.legend(handles=custom_lines, bbox_to_anchor=(1.0, 0.75))


plot_points(x_train, y_train, x_test, y_test)

TypeError: list indices must be integers or slices, not float